In [1]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import linear_model
import gzip
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import string
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import random

In [2]:
def show_acc_by_stars(preds, actual):
    distri = Counter(actual)
    five = 0
    four = 0
    three = 0
    two = 0
    one = 0
    for x,y in zip(preds,actual):
        if x ==y:
            if y ==5.0:
                five+=1
            elif y ==4.0:
                four+=1
            elif y==3.0:
                three+=1
            elif y==2.0:
                two+=1
            else:
                one+=1
                
    one_text = "1 :   " + str(one/distri[1.0])
    two_text = "2 :   " + str(two/distri[2.0])
    three_text = "3 :   " + str(three/distri[3.0])
    four_text = "4 :   " + str(four/distri[4.0])
    five_text = "5 :   " + str(five/distri[5.0])
    overall_acc = "overall accuracy:       " + str(accuracy_score(preds,actual))
    
    print(one_text+'\n'+two_text+'\n'+three_text+'\n'+four_text+'\n' + five_text + '\n'+overall_acc)                

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [6]:
f = gzip.open("Digital_Music.json.gz")
dataset = []
for l in f:
    dataset.append(eval(l,{"true":True,"false":False,"null":None}))

# XGBoost

In [9]:
selected_data = []
for d in dataset:
    if 'reviewText' not in d:
        continue
    if 'overall' not in d:
        d['overall'] = 5.0
    selected_data.append({'reviewText':d['reviewText'], 'overall': d['overall']})

In [11]:
df = pd.DataFrame(selected_data)

In [13]:
df.head(10)

,reviewText,overall
0,This is a great cd full of worship favorites!!...,5.0
1,"So creative! Love his music - the words, the ...",5.0
2,"Keith Green, gone far to early in his carreer,...",5.0
3,Keith Green had his special comedy style of Ch...,5.0
4,Keith Green / So you wanna go back to Egypt......,5.0
5,"Clever,inspired and moving. This is a great al...",5.0
6,This tape can hardly be understood and it was ...,1.0
7,Keith Green is a bit of legend in some Christi...,5.0
8,Pleased,5.0
9,Buy the CD. Do not buy the MP3 album. Downlo...,1.0


In [15]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')
text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

[nltk_data] Error loading stopwords: <urlopen error EOF occurred in
[nltk_data]     violation of protocol (_ssl.c:1129)>


In [16]:
import re

def preprocess(text, stem=False):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [17]:
df.reviewText = df.reviewText.apply(lambda x : preprocess(x))

In [23]:
df = df[:100000]

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

text_transformer = CountVectorizer()

X_text = text_transformer.fit_transform(df['reviewText'])

X_train, X_test, y_train, y_test = train_test_split(X_text, df['overall'], test_size=0.2)

model = xgb.XGBRegressor()
model.fit(X_train, y_train)

n_folds = 10

score = cross_val_score(model, X_train, y_train, cv=n_folds)

array([0.22552753, 0.25434841, 0.27109127, 0.26123587, 0.23214322,
       0.22596947, 0.24817317, 0.26903428, 0.22531921, 0.26377041])

In [29]:
sum(score)/10

0.24766128435570273

In [25]:
y_pred = model.predict(X_test)

In [36]:
y_test[:10]

18877    5.0
2481     5.0
48914    4.0
41022    4.0
40746    5.0
44164    5.0
12980    5.0
25540    2.0
32759    5.0
49824    5.0
Name: overall, dtype: float64

In [26]:
y_pred[:10]

array([4.7574635, 4.3767195, 4.934208 , 1.4054102, 2.1225338, 4.689172 ,
       4.919828 , 4.881586 , 4.9714627, 4.7912602], dtype=float32)

In [27]:
mse = mean_squared_error(y_pred, y_test)
mse

0.5401286898821185

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

text_transformer = TfidfVectorizer()

X_text = text_transformer.fit_transform(df['reviewText'])

X_train, X_test, y_train, y_test = train_test_split(X_text, df['overall'], test_size=0.2)

model = xgb.XGBRegressor()
model.fit(X_train, y_train)

n_folds = 10

score = cross_val_score(model, X_train, y_train, cv=n_folds)

In [31]:
sum(score)/10

0.2669533219705811

In [32]:
y_pred = model.predict(X_test)

In [38]:
y_test[:10]

18877    5.0
2481     5.0
48914    4.0
41022    4.0
40746    5.0
44164    5.0
12980    5.0
25540    2.0
32759    5.0
49824    5.0
Name: overall, dtype: float64

In [37]:
y_pred[:10]

array([4.6916323, 4.8824606, 4.841428 , 3.9187117, 4.6114726, 4.7877674,
       4.7366805, 4.500876 , 5.0250072, 4.890143 ], dtype=float32)

In [33]:
mse = mean_squared_error(y_pred, y_test)
mse

0.5048193823541367